In [1]:
#
# a program to make .fsgd file for fREEsURFER-script fs_glm.sh from .xlsx files
#
import pandas as pd
import os
IMPACT_CWD=os.path.expanduser("~")+"/Downloads/impact/" # working directory
#print(IMPACT_CWD)
bl_book = pd.ExcelFile(IMPACT_CWD+'BHQ_Research_2018_01_Access用_20200108.xlsx') # base lien file
fu_book = pd.ExcelFile(IMPACT_CWD+'BHQ_Research_2018_02_Access用_20200108.xlsx') # follow up file

# 手動で取り除くIDを指定
IDsQCfailed=["MR0011_0742","MR0011_0770","MR0011_0810"] # QCでNGのID
IDsQCfailed=IDsQCfailed+["MR0011_0681","MR0011_0787","MR0011_0793"] # niftiファイルの無いID
IDsQCfailed=IDsQCfailed+["MR0011_0761"] # follow up画像の無いID
print("取り除くID:",IDsQCfailed)

取り除くID: ['MR0011_0742', 'MR0011_0770', 'MR0011_0810', 'MR0011_0681', 'MR0011_0787', 'MR0011_0793', 'MR0011_0761']


In [2]:
# read base line
bl_sheet_name = bl_book.sheet_names
bl_sheet_df = bl_book.parse(bl_sheet_name[0],skiprows=3,skipfooter=5)
bl_sheet_df=bl_sheet_df[bl_sheet_df[bl_sheet_df.columns[1]].str.contains("",na=False)] # id exists
#print("Sheet name:", bl_sheet_name[0])
bl_sheet_df=bl_sheet_df.loc[:,["ImPACT_ID","Arm_Type_01","Age_01","Gender_12_01","MRI_Imaging_Date_01",\
                               "FABHQ_01","FABHQ_DMN_01","FABHQ_CEN_01","FABHQ_SN_01",\
                               "GMBHQ_01","GMBHQ_DMN_01","GMBHQ_CEN_01","GMBHQ_SN_01",\
                               "SPA_Unrelated_SUM_01","TMT_PartA_01_01","TMT_PartB_01_01",\
                               "MMSEJ_SUM_01","CESD_Score_01"]] # only base lines have MMSE
#print(bl_sheet_df.shape)
#print(bl_sheet_df.tail(5))

# read follow up
fu_sheet_name = fu_book.sheet_names
fu_sheet_df=fu_book.parse(fu_sheet_name[0],skiprows=3,skipfooter=5)
fu_sheet_df=fu_sheet_df[fu_sheet_df[fu_sheet_df.columns[1]].str.contains("",na=False)] # id exists
#print("Sheet name:", fu_sheet_name[0])
fu_sheet_df=fu_sheet_df.loc[:,["ImPACT_ID","Arm_Type_02","Age_02","Gender_12_02","MRI_Imaging_Date_02",\
                               "FABHQ_02","FABHQ_DMN_02","FABHQ_CEN_02","FABHQ_SN_02",\
                               "GMBHQ_02","GMBHQ_DMN_02","GMBHQ_CEN_02","GMBHQ_SN_02",\
                               "SPA_Unrelated_SUM_01","TMT_PartA_01_02","TMT_PartB_01_02"]] #,"MMSEJ_SUM_01","CESD_Score_01"
#print(fu_sheet_df.shape)
#print(fu_sheet_df.tail(5))

# merge baseline and followup
bl_fu_sheet_df=pd.merge(bl_sheet_df, fu_sheet_df, left_on=bl_sheet_df.columns[0], right_on=fu_sheet_df.columns[0], how="inner", suffixes = ["_bl", "_fu"])
bl_fu_sheet_df.columns=bl_fu_sheet_df.columns.str.replace(" ","_")
bl_fu_sheet_df.columns=bl_fu_sheet_df.columns.str.replace("\n","_")

In [3]:
# remove IDs whose QC failed
bl_fu_sheet_df=bl_fu_sheet_df[~bl_fu_sheet_df["ImPACT_ID"].isin(IDsQCfailed)]

# remove IDs depressed
#print("IDs depressed:\n",bl_fu_sheet_df[bl_fu_sheet_df["CESD_Score_01"]>=16][["ImPACT_ID","Arm_Type_01","CESD_Score_01"]])
#bl_fu_sheet_df=bl_fu_sheet_df[bl_fu_sheet_df["CESD_Score_01"]<16]

# make data frame
df_result=bl_fu_sheet_df.loc[:,["ImPACT_ID","Arm_Type_01","Age_01", "Age_02"]]
df_result["fID"]="V_"+df_result["ImPACT_ID"]+"_"+df_result["Arm_Type_01"]
"""
df_result["SPA_Unrelated_SUM_01"]=df_result["SPA_Unrelated_SUM_01_bl"]
df_result["SPA_Unrelated_SUM_02"]=df_result["SPA_Unrelated_SUM_01_fu"]
df_result=df_result.drop(columns=["SPA_Unrelated_SUM_01_bl","SPA_Unrelated_SUM_01_fu"])
df_result=df_result.drop(columns=["MRI_Imaging_Date_01", "MRI_Imaging_Date_02"])
"""
df_result=df_result.set_index("fID")

In [4]:
# make .mat file for FSL TBSS
ctrl_grouplist=["AR"]
intrv_grouplist=["AS","AT","AU"]
#impact_grouplist=["AR","AS","AT","AU"]
suffixlist=["_01","_02"]
scorelist=["SPA_Unrelated_SUM_01_bl","TMT_PartA_01_01","TMT_PartB_01_01","MMSEJ_SUM_01",\
           "SPA_Unrelated_SUM_01_fu","TMT_PartA_01_02","TMT_PartB_01_02"]

In [5]:
def make_tbssfile(temp_filenamebody, temp_dfvarname, temp_varname, suffixlist): # subroutine
    for sfx in suffixlist:
        for ss in intrv_grouplist:
            classlist=[ctrl_grouplist[0], ss]

            data1=[]
            data2=[]
            data3=[]
            for temp_line in df_result.itertuples():
                temp_armtype=temp_line[df_result.columns.get_loc("Arm_Type_01")+1] # 1 for .index
                if temp_armtype in classlist:
                    str_dfvar=str(temp_line[df_result.columns.get_loc(temp_dfvarname+sfx)+1]) # 1 for .index
                    if str_dfvar.replace('.', '').replace('-', '').isnumeric(): # ignore NaN
                        if temp_armtype==classlist[0]:
                            is_group1=1
                        else:
                            is_group1=0
                        if temp_armtype==classlist[1]:
                            is_group2=1
                        else:
                            is_group2=0
                        data1.append(is_group1)
                        data2.append(is_group2)
                        data3.append(int(str_dfvar))

            filename=IMPACT_CWD+temp_filenamebody+ss+sfx+".mat.txt"
            outfile=open(filename, "wt")
            """
            outfile.write("/NumWaves "+str(3)+"\n")
            outfile.write("/NumPoints "+str(len(data1))+"\n")
            outfile.write("/PPheights "+str(max(max(data1), 0)-min(min(data1), 0))+" "+str(max(max(data2), 0)-min(min(data2), 0))+" "+str(max(max(data3), 0)-min(min(data3), 0))+"\n")
            outfile.write("/Matrix"+"\n")
            outfile.write("\n")
            """
            for ii in range(len(data1)):
                outfile.write(str(data1[ii])+" "+str(data2[ii])+" "+str(data3[ii])+"\n")
            outfile.close()

            filename=IMPACT_CWD+temp_filenamebody+ss+sfx+".con.txt"
            outfile=open(filename, "wt")
            """
            outfile.write("/ContrastName1	"+classlist[0]+" > "+classlist[1]+"\n")
            outfile.write("/ContrastName2	"+classlist[0]+" < "+classlist[1]+"\n")
            outfile.write("/ContrastName3	pos age corr "+"\n")
            outfile.write("/ContrastName4	neg age corr "+"\n")
            outfile.write("/NumWaves	"+str(3)+"\n")
            outfile.write("/NumContrasts	"+str(4)+"\n")
            #outfile.write("/PPheights		"+str(1.023956e+00)+" "+str(1.023956e+00)+" "+str(2.820455e+01)+" "+str(2.820455e+01)+"\n")
            #outfile.write("/RequiredEffect		"+str(0.834)+" "+str(0.834)+" "+str(1.486)+" "+str(1.486)+"\n")
            #outfile.write("\n")
            #outfile.write("/Matrix"+"\n")
            """
            outfile.write(str(1)+" "+str(-1)+" "+str(0)+"\n")
            outfile.write(str(-1)+" "+str(1)+" "+str(0)+"\n")
            outfile.write(str(0)+" "+str(0)+" "+str(1)+"\n")
            outfile.write(str(0)+" "+str(0)+" "+str(-1)+"\n")
            outfile.close()
    return

temp_filenamebody="design_g2v1_"
temp_dfvarname="Age"
#temp_title="group_age_"
temp_varname="age"
temp_suffixlist=suffixlist
make_tbssfile(temp_filenamebody, temp_dfvarname, temp_varname, temp_suffixlist) # call subroutine